In [2]:
# Importing necessary libraries
import h5py  # h5py is used for handling HDF5 files
import tensorflow as tf  # TensorFlow is a library for machine learning
from tensorflow.keras.models import model_from_json  # This imports the function to load Keras models

In [ ]:
# Load the Keras model from an H5 file
model = tf.keras.models.load_model('/content/drive/MyDrive/Ph5 y json/E15model_sin_finetunningrEfficientnetv2b0.h5')

In [ ]:
# Display the summary of the model's architecture
model.summary()  # This shows the structure and layers of the model

In [5]:
# Importing the ImageDataGenerator class from Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importing tools from scikit-learn for evaluating the model
from sklearn.metrics import classification_report, confusion_matrix  # Used for generating performance metrics (report and confusion matrix)


In [ ]:
# Create an instance of ImageDataGenerator for image rescaling (normalization)
datagen = ImageDataGenerator(rescale=1./255)  # Rescales image pixel values to the range [0, 1]

# Create a generator that will load images from the specified directory
test_generator = datagen.flow_from_directory(
    '/content/DATASET evauaciOn/resizedtestpatches',  # Path to the test dataset
    target_size=(512, 512),  # Resizes all images to the target size (512X512 pixels)
    batch_size=32,  # Number of images to process in each batch
    class_mode='categorical',  # Specifies the format of the labels (for multi-class classification)
    shuffle=False  # Don't shuffle the data, as this is for evaluation on the test set
)

In [9]:
# Clear the Keras session to free up memory
tf.keras.backend.clear_session()  # This is used to reset the Keras backend, clearing the session and freeing memory


In [10]:
# Compile the model with the specified optimizer, loss function, and evaluation metrics
model.compile(optimizer='adam',  # Adam optimizer is used for efficient gradient descent
              loss='categorical_crossentropy',  # Loss function for multi-class classification problems
              metrics=['accuracy'])  # Metric to track during training and evaluation


In [ ]:
import numpy as np
# Evaluate the model on the test data and get the results for each metric
results = model.evaluate(test_generator)  # Evaluates the model on the test set and returns the loss and metrics

# Print the result for each metric (e.g., accuracy, loss)
for metric_name, value in zip(model.metrics_names, results):
    print(f'{metric_name}: {value}')  # Prints the name of each metric and its corresponding value

# Obtain predictions in small batches to avoid memory issues
y_pred = []  # List to store predicted labels
y_true = []  # List to store true labels

# Loop through the test set and make predictions in batches
for i in range(len(test_generator)):
    X, y = test_generator[i]  # Get a batch of data (images and labels)
    y_pred.extend(np.argmax(model.predict(X), axis=1))  # Predict the class with the highest probability
    y_true.extend(np.argmax(y, axis=1))  # Get the true labels (index of the highest value)

# Print classification report (precision, recall, f1-score)
print(classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))

# Print confusion matrix to evaluate the performance of the model
print(confusion_matrix(y_true, y_pred))  # Confusion matrix shows the number of correct/incorrect predictions

In [12]:
# Importing the Seaborn library for data visualization (for plotting)
import seaborn as sns  # Seaborn is used for creating statistical graphics in Python

# Importing Matplotlib for additional plotting functionalities
import matplotlib.pyplot as plt  # Matplotlib is used for generating visualizations like graphs and charts


In [13]:
# Get the shape of the confusion matrix to see its dimensions
confusion_matrix_shape = confusion_matrix(y_true, y_pred).shape  # Returns the dimensions (shape) of the confusion matrix


In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))  # Adjust the figure size if needed
sns.heatmap(confusion_matrix(y_true, y_pred),  # Plot the confusion matrix
            annot=True,  # Annotates each cell with its numeric value
            fmt='d',  # Format the annotation as an integer
            cmap='Blues',  # Use a blue color map for the heatmap
            xticklabels=test_generator.class_indices.keys(),  # Set x-axis labels to class names
            yticklabels=test_generator.class_indices.keys())  # Set y-axis labels to class names

plt.xlabel('Prediction')  # Label for the x-axis (Predicted classes)
plt.ylabel('True values')  # Label for the y-axis (True classes)
plt.title('Confusion Matrix')  # Title of the plot
plt.show()  # Display the plot

In [ ]:
import json  # Importing the JSON library for parsing JSON files
import matplotlib.pyplot as plt  # Importing Matplotlib for plotting graphs
import numpy as np  # Importing NumPy for numerical operations

# List of paths to JSON files (one per fold in cross-validation)
json_files = [
    '/content/drive/MyDrive/Ph5 y json/E15history_sin_finetunningrEfficientnetv2b0.json',  # Path to the JSON file containing history data
]

# Initializing lists to store metrics from each fold
accuracy_train = []  # List to store training accuracy
accuracy_val = []  # List to store validation accuracy
loss_train = []  # List to store training loss
loss_val = []  # List to store validation loss

# Load and process each JSON file
for file in json_files:
    with open(file, 'r') as f:  # Open the JSON file in read mode
        history = json.load(f)  # Load the JSON content into a dictionary

        # Extend the lists with the data from the JSON
        accuracy_train.extend(history['accuracy'])  # Training accuracy
        accuracy_val.extend(history['val_accuracy'])  # Validation accuracy
        loss_train.extend(history['loss'])  # Training loss
        loss_val.extend(history['val_loss'])  # Validation loss

# Convert metrics to NumPy arrays for easier analysis
accuracy_train = np.array(accuracy_train)
accuracy_val = np.array(accuracy_val)
loss_train = np.array(loss_train)
loss_val = np.array(loss_val)

# Plot the metrics for each epoch
epochs = np.arange(1, accuracy_train.shape[0] + 1)  # Create an array of epoch numbers

plt.figure(figsize=(12, 6))  # Set the size of the figure

# Plot training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, accuracy_train, label='Train Accuracy', color='blue')  # Training accuracy curve
plt.plot(epochs, accuracy_val, label='Validation Accuracy', color='green', linestyle='--')  # Validation accuracy curve
plt.title('Accuracy per Epoch')  # Title for the accuracy plot
plt.xlabel('Epoch')  # X-axis label for epochs
plt.ylabel('Accuracy')  # Y-axis label for accuracy
plt.legend()  # Show the legend

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss_train, label='Train Loss', color='red')  # Training loss curve
plt.plot(epochs, loss_val, label='Validation Loss', color='orange', linestyle='--')  # Validation loss curve
plt.title('Loss per Epoch')  # Title for the loss plot
plt.xlabel('Epoch')  # X-axis label for epochs
plt.ylabel('Loss')  # Y-axis label for loss
plt.legend()  # Show the legend

plt.tight_layout()  # Adjust the layout for better spacing
plt.show()  # Display the plots

# Global accuracy statistics (mean and standard deviation)
mean_accuracy_train = np.mean(accuracy_train)  # Mean training accuracy
std_accuracy_train = np.std(accuracy_train)  # Standard deviation of training accuracy
mean_accuracy_val = np.mean(accuracy_val)  # Mean validation accuracy
std_accuracy_val = np.std(accuracy_val)  # Standard deviation of validation accuracy

# Print the global accuracy statistics
print(f"Mean training accuracy: {mean_accuracy_train:.4f} ± {std_accuracy_train:.4f}")
print(f"Mean validation accuracy: {mean_accuracy_val:.4f} ± {std_accuracy_val:.4f}")

# Global loss statistics (mean and standard deviation)
mean_loss_train = np.mean(loss_train)  # Mean training loss
std_loss_train = np.std(loss_train)  # Standard deviation of training loss
mean_loss_val = np.mean(loss_val)  # Mean validation loss
std_loss_val = np.std(loss_val)  # Standard deviation of validation loss

# Print the global loss statistics
print(f"Mean training loss: {mean_loss_train:.4f} ± {std_loss_train:.4f}")
print(f"Mean validation loss: {mean_loss_val:.4f} ± {std_loss_val:.4f}")


In [17]:
from sklearn.metrics import roc_curve, auc

In [ ]:
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

In [19]:
from sklearn.preprocessing import label_binarize

# Binarize the labels for multi-class ROC calculation
y_true_bin = label_binarize(y_true, classes=list(range(len(test_generator.class_indices))))
n_classes = y_true_bin.shape[1]

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], Y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), Y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
#ROC PLOT
plt.figure()
colors = ['aqua', 'darkorange', 'cornflowerblue', 'red', 'green', 'purple', 'brown', 'pink', 'gray', 'olive'] # Add more colors if needed
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
                   ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) for each class')
plt.legend(loc="lower right")
plt.show()